# Limpieza V. Sklearn.

In [31]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sidetable as stb
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer

pd.options.display.max_columns = None
pd.options.display.max_rows = None


In [32]:
df = pd.read_csv('datos/Limpieza-3.csv', index_col=0)
df.head()

,year,type,country,activity,age,species,month,fatal,sex
0,2018,Boating,usa,Paddling,57.0,White Shark,Jun,N,F
1,2018,Unprovoked,brazil,Swimming,18.0,Tiger Shark,Jun,Y,M
2,2018,Unprovoked,usa,Walking,15.0,Bull Shark,May,N,M
3,2018,Provoked,australia,Feeding sharks,32.0,Grey Shark,May,N,M
4,2018,Invalid,england,Fishing,21.0,Unspecified,May,N,M


**1. Es el momento de eliminar los nulos:**

- Reemplazad los valores nulos de la columna age por la media de la edad usando el método SimpleImputer.

In [33]:
numericas = df.select_dtypes(include=np.number)
numericas.head()

,year,age
0,2018,57.0
1,2018,18.0
2,2018,15.0
3,2018,32.0
4,2018,21.0


In [34]:
#calculamos la media con el método SimpleImputer
imputer_mean = SimpleImputer(strategy='mean', missing_values=np.nan)

In [35]:
#aplicamos la instancia sobre las variables numéricas
imputer_mean.fit(numericas)
# transformamos nuestros datos numéricos
imputer_mean.transform(numericas)

array([[2018.        ,   57.        ],
       [2018.        ,   18.        ],
       [2018.        ,   15.        ],
       ...,
       [1997.        ,   29.20475561],
       [1997.        ,   29.20475561],
       [1997.        ,   15.        ]])

In [36]:
#pasamos a dataframe el array del bloque anterior:
df_numericas = pd.DataFrame(imputer_mean.transform(numericas), columns = numericas.columns)
df_numericas.head()

,year,age
0,2018.0,57.0
1,2018.0,18.0
2,2018.0,15.0
3,2018.0,32.0
4,2018.0,21.0


In [37]:
for col in df_numericas.columns:
    if df_numericas[col].isnull().sum() == 0:
        print(f'En la columna {col} ya no hay casos nulos.')
    else:
        print(f'En la columna {col} hay {df_numericas[col].isnull().sum()} casos nulos.')

En la columna year ya no hay casos nulos.
En la columna age ya no hay casos nulos.


In [38]:
#calculamos la media de las columnas para comprobar luego si el reemplazo ha funcionado
for col in df_numericas.columns:
    print(f'La media de {col} es {round(df[col].mean())}')
    print("------------------------------------------------------------")

La media de year es 1991
------------------------------------------------------------
La media de age es 29
------------------------------------------------------------


In [39]:
#extraemos los valores únicos de cada columna para comprobar que ha funcionado:
for col in df_numericas.columns:
    print(f"En la columna {col} tenemos los siguientes valores unicos")
    print(pd.DataFrame(df[col].value_counts()).head())
    print("------------------------------------------------------------")

En la columna year tenemos los siguientes valores unicos
      year
1997   192
2015    68
2007    57
2012    54
2002    53
------------------------------------------------------------
En la columna age tenemos los siguientes valores unicos
      age
25.0   82
19.0   71
18.0   61
20.0   60
15.0   57
------------------------------------------------------------


In [40]:
for col in df_numericas.columns:
    if df_numericas[col].isnull().sum() == 0:
        print(f'En la columna {col} ya no hay casos nulos.')
    else:
        print(f'En la columna {col} hay {df_numericas[col].isnull().sum()} casos nulos.')

En la columna year ya no hay casos nulos.
En la columna age ya no hay casos nulos.


- Reemplazad los valores nulos de la columna sex por la moda, usando el método SimpleImputer.  

    💡 Pista 💡 La moda en este tipo de aproximación se indica como most_frequent.

In [41]:
#conteamos los valores de la columna, incluyendo los nulos.
df['sex'].value_counts(dropna=False)

M      1435
F       223
NaN      14
Name: sex, dtype: int64

In [42]:
print(f'La columna Sexo tiene {df["sex"].isnull().sum()} casos nulos.')

La columna Sexo tiene 14 casos nulos.


In [43]:
#asignamos el valor constante por el que queremos reemplazar los nulos. En este caso, la moda.
imputer_mode = SimpleImputer(strategy='most_frequent', missing_values=np.nan)

In [44]:
#Calculamos cuál es la moda de nuestra columna para comprobar que el reemplazo ha funcionado cuando lo apliquemos:
print(f'La moda de la columna Sexo es {df["sex"].mode()[0]}.')

La moda de la columna Sexo es M.


In [45]:
#aplicamos la instancia y transformamos sobre la variable categórica sex en un array:
imputer_mode.fit(df[["sex"]])
imputer_mode.transform(df[["sex"]])

array([['F'],
       ['M'],
       ['M'],
       ...,
       ['M'],
       ['M'],
       ['M']], dtype=object)

In [46]:
#lo pasamos a DataFrame
df_sex = pd.DataFrame(imputer_mode.transform(df[["sex"]]), columns = ["Sexo"])
df_sex.head()


,Sexo
0,F
1,M
2,M
3,M
4,M


In [47]:
#comprobamos que los valores nulos no están presentes en el nuevo DF.

print(f'En la columna Sexo del nuevo DataFrame tenemos {df_sex["Sexo"].isnull().sum()} casos nulos.')
print(f'En la columna Sexo del nuevo DataFrame, tenemos {df_sex.value_counts()[0]} hombres y {df_sex.value_counts()[1]} mujeres.')

En la columna Sexo del nuevo DataFrame tenemos 0 casos nulos.
En la columna Sexo del nuevo DataFrame, tenemos 1449 hombres y 223 mujeres.


In [48]:
#reemplazamos los valores:
df['sex'].replace(df['sex'].values, df_sex['Sexo'].values, inplace=True)
df['sex'].value_counts(dropna=False)

M    1449
F     223
Name: sex, dtype: int64

In [49]:
nulo_sexo = df['sex'].isnull().sum()

if nulo_sexo == 0:
    print(f'En la columna "Sexo" ya no hay casos nulos.')
else:
    print(f'En la columna "Sexo" hay {nulo_sexo} casos nulos.')

En la columna "Sexo" ya no hay casos nulos.


- Reemplazad los valores nulos de la columna type por el valor más frecuente (la moda) con el método SimpleImputer.

In [50]:
#vamos a explorar los casos nulos de cada columna:
for col in df:
    if df[col].isnull().sum() != 0:
        print(f'La columna {col} tiene {df[col].isnull().sum()} casos nulos.')
    else:
        print(f'La columna {col} no tiene casos nulos.')

La columna year no tiene casos nulos.
La columna type no tiene casos nulos.
La columna country tiene 10 casos nulos.
La columna activity tiene 31 casos nulos.
La columna age tiene 158 casos nulos.
La columna species tiene 126 casos nulos.
La columna month tiene 181 casos nulos.
La columna fatal tiene 99 casos nulos.
La columna sex no tiene casos nulos.


Type no tiene nulos, por lo que aplicaremos el método solicitado en el enunciado con la columna "Country"

In [51]:
#extraemos las frecuencias con el parámetro thresh ya que son demasiados países con muy pocos ataques:
df.stb.freq(['country'], thresh=85)

,country,count,percent,cumulative_count,cumulative_percent
0,usa,741,44.584838,741,44.584838
1,australia,343,20.637786,1084,65.222623
2,south africa,234,14.079422,1318,79.302046
3,bahamas,29,1.744886,1347,81.046931
4,new zealand,21,1.263538,1368,82.310469
5,reunion,17,1.022864,1385,83.333333
6,papua new guinea,16,0.962696,1401,84.296029
7,others,261,15.703971,1662,100.000000


In [52]:
#extraemos el número de nulos que tiene la columna:
print(f'La columna Country tiene {df["country"].isnull().sum()} casos nulos.')

La columna Country tiene 10 casos nulos.


In [53]:
#asignamos el valor constante por el que queremos reemplazar los nulos. En este caso, la moda.
imputer_mode_c =  SimpleImputer(strategy='most_frequent', missing_values=np.nan)

In [54]:
#Calculamos cuál es la moda de nuestra columna para comprobar que el reemplazo ha funcionado cuando lo apliquemos:
print(f'La moda de la columna País es {(df["country"].mode()[0]).upper()}.')

La moda de la columna País es USA.


In [55]:
imputer_mode_c.fit(df[['country']])
imputer_mode_c.transform(df[['country']])

array([['usa'],
       ['brazil'],
       ['usa'],
       ...,
       ['usa'],
       ['panama'],
       ['ceylon (sri lanka)']], dtype=object)

In [56]:
#generamos el array y lo volcamos directamente en el DF:
df_country = pd.DataFrame(imputer_mode_c.transform(df[['country']]), columns=['Pais'])
df_country.head()

,Pais
0,usa
1,brazil
2,usa
3,australia
4,england


In [57]:
#comprobamos que los valores nulos no están presentes en el nuevo DF:
print(f'Hay {df_country["Pais"].isnull().sum()} casos nulos en el nuevo DF.')


Hay 0 casos nulos en el nuevo DF.


In [58]:
#reemplazamos los valores:
df['country'].replace(df['country'].values, df_country['Pais'].values, inplace=True)

#comprobamos que se han aplicado los cambios en los valores de la moda (USA)
df.stb.freq(['country'], thresh=85)

,country,count,percent,cumulative_count,cumulative_percent
0,usa,751,44.916268,751,44.916268
1,australia,343,20.514354,1094,65.430622
2,south africa,234,13.995215,1328,79.425837
3,bahamas,29,1.734450,1357,81.160287
4,new zealand,21,1.255981,1378,82.416268
5,reunion,17,1.016746,1395,83.433014
6,papua new guinea,16,0.956938,1411,84.389952
7,others,261,15.610048,1672,100.000000


In [59]:
if df["country"].isnull().sum() == 0:
    print(f'En la columna "country" ya no hay casos nulos.')
else:
    print(f'En la columna "country" hay {df["country"].isnull().sum()} casos nulos.')

En la columna "country" ya no hay casos nulos.


- Utilizad el método KNN Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [60]:
#volvimos a abrir el DF para reiniciar el reemplazo de nulos de apartados anteriores:
df = pd.read_csv('datos/Limpieza-3.csv', index_col=0)
df.head()

,year,type,country,activity,age,species,month,fatal,sex
0,2018,Boating,usa,Paddling,57.0,White Shark,Jun,N,F
1,2018,Unprovoked,brazil,Swimming,18.0,Tiger Shark,Jun,Y,M
2,2018,Unprovoked,usa,Walking,15.0,Bull Shark,May,N,M
3,2018,Provoked,australia,Feeding sharks,32.0,Grey Shark,May,N,M
4,2018,Invalid,england,Fishing,21.0,Unspecified,May,N,M


In [61]:
#volvemos a definir también el df con las variables numéricas, dada su depuración anterior:
numericas = df.select_dtypes(include=np.number)
numericas.head()

,year,age
0,2018,57.0
1,2018,18.0
2,2018,15.0
3,2018,32.0
4,2018,21.0


In [62]:
#comprobamos el número de nulos de cada columna:
for col in numericas:
    if numericas[col].isnull().sum() != 0:
        print(f'En la columna {col} hay {numericas[col].isnull().sum()} casos nulos.')
    else:
        print(f'En la columna {col} no hay casos nulos.')

En la columna year no hay casos nulos.
En la columna age hay 158 casos nulos.


In [63]:
imputerKNN = KNNImputer(n_neighbors=5)
imputerKNN.fit(numericas)

KNNImputer()

In [64]:
#generamos el array y lo volcamos directamente en el DF:
df_knn_numericas = pd.DataFrame(imputerKNN.transform(numericas), columns=numericas.columns)
df_knn_numericas.head()

,year,age
0,2018.0,57.0
1,2018.0,18.0
2,2018.0,15.0
3,2018.0,32.0
4,2018.0,21.0


- Utilizad el método Iterative Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [65]:
knn_cols = df_knn_numericas.columns
knn_cols

Index(['year', 'age'], dtype='object')

In [66]:
#borramos las columnas del DF original que hemos modificado en el df_knn_numericas:
df.drop(knn_cols, axis=1, inplace=True)

In [67]:
#reemplazamos las columnas que hemos borrado por aquellas creadas en el DF df_knn_numericas:
df[knn_cols] = df_knn_numericas

In [68]:
#comprobamos si se han borrado los nulos:
for col in df.select_dtypes(include=np.number):
    if df[col].isnull().sum() != 0:
        print(f'En la columna {col} hay {df[col].isnull().sum()} casos nulos.')
    else:
        print(f'En la columna {col} no hay casos nulos.')

En la columna year no hay casos nulos.
En la columna age no hay casos nulos.


- ¿Podríais explicar qué diferencia hay entre estos dos últimos métodos?

Tanto el SimpleImputer como el KNNImputer son modelos pasados en métodos de imputación, es decir, métodos que reemplazan los datos nulos al crear una matriz de datos completa a partir de correlaciones entre valores de la base original (conocidos como "vecinos"). 

La **diferencia entre SimpleImputer y KNNImputer** es que ambos usan diferentes estrategias para calcular estos valores por los que reemplazar a los NA:

Mientras que SimpleImputer ----, KNNImputer mide la distancia entre cada punto (valores),


**2. Guardad el csv para seguir trabajando con el en los siguientes ejercicios de pair:**

Como hemos vuelto a invocar el DF, hemos guardado los cambios realizados anteriormente para que no se queden solo los cambios realizados a las columnas numéricas:

In [69]:
#https://stackoverflow.com/a/54174184
raise SystemExit("Stop right there!")

SystemExit: Stop right there!

c:\Users\PcCom\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
df.to_csv('datos/Limpieza-5.csv')